# RVE search

We want to look in modules 3 and 6 (new numbering) for RVE elements. We'll do this by plotting the forward and reverse complements of all the motifs in each module, next to those of the evening elements were interested in.

In [34]:
import os
from os import listdir
from os.path import splitext, basename
import logomaker
import pandas as pd
import matplotlib.pyplot as plt

## Defining the functions
We're going to adapt the code from Ann's motif_merge_functions notebook.

In [56]:
def load_binding(path, remove=True):
    # path should be to a file in the HOMER motif format, which is tab-separated
    # the first row is a header (ignored)
    # the rest are the position weights for A, C, G, and T
    # "remove" (the fifth column) is empty; if your file doesn't have this, set remove to False
    cols = ["A","C","G","T"]
    if remove == True:
        cols.append("remove")
    df = pd.read_csv(path, sep="\t", header=None,skiprows=1,names=cols)
    if remove == True:
        df = df.drop("remove", axis=1)
    df.reset_index(names="position", inplace=True)
    return df

In [3]:
def rev_comp(df):
    #reverse order first:
    new = df.copy()
    new.position = -1*new.position
    new.sort_values("position", ascending=True, inplace=True)
    new.position = -1*new.position
    new.reset_index(inplace=True, drop=True)
    new.position=new.index
    #complement by swapping A with T, G with C
    new.rename({col: col+"_old" for col in new.columns if col != "position"}, axis=1, inplace=True)
    new["A"] = new.T_old
    new["C"] = new.G_old
    new["G"] = new.C_old
    new["T"] = new.A_old
    new = new[[col for col in new.columns if "old" not in col]]
    return new

Now the one we need to actually change is the `make_logo` function. Currently it plots anything you give it on a new figure, but we want to be able to streamline this.

In [5]:
def make_logo(motif, ax):
    # motif is a df with the columns of "A" "C" "G" and "T"
    # optional columns include "position"
    if "position" in motif.columns:
        motif.set_index("position", inplace=True)
    num_bases = int(motif.index.max())
    print(num_bases)
    motif_t = logomaker.transform_matrix(motif, from_type="probability", to_type="information")
    logo = logomaker.Logo(motif_t, figsize=(num_bases, 2.5), ax=ax)

In [57]:
def plot_multiple_logos(search_for, search_against):
    """
    Plot forward and reverse complements of motifs in a way that allows
    direct comparison.

    parameters:
        search_for, list of str: paths of motif files containingthe motifs of
            interest that we're looking for
        search_against, list of str: paths of motif files from a given module
            to search through for search_for motifs

    returns: None
    """
    # Make two cols for every motif to search for plus to for the search against
    num_cols = len(search_for)*2 + 2
    num_rows = len(search_against)

    # Get the forward and reverse complements of the search_fors
    search_for_data = {}
    for_names = []
    for s in search_for:
        name = splitext(basename(s))[0]
        for_names.append(name)
        forward = load_binding(s, remove=False)
        reverse = rev_comp(forward)
        search_for_data[name] = {'forward': forward, 'reverse': reverse}

    # Do the same for the search against
    search_against_data = {}
    against_names = []
    for s in search_against:
        name = splitext(basename(s))[0]
        against_names.append(name)
        forward = load_binding(s, remove=True)
        reverse = rev_comp(forward)
        search_against_data[name] = {'forward': forward, 'reverse': reverse}

    # Now plot
    fig, axs = plt.subplots(num_rows, num_cols)

    # The first len(search_for)*2 cols will always have the same plots in every row
    for i, ax_row in enumerate(axs):
        for j, ax in enumerate(ax_row):
            if (j % 2 == 0) and (j < len(search_for)*2): # Then it's a search_for forward plot
                motif = search_for_data[for_names[j]]['forward']
                make_logo(motif.copy())
                ax.set_title(f'{for_names[j]} forward')
            elif (j % 2 != 0) and (j < len(search_for)*2): # Then it's a search_for reverse plot
                motif = search_for_data[for_names[j]]['reverse']
                make_logo(motif.copy())
                ax.set_title(f'{for_names[j]} reverse')
            elif (j % 2 == 0) and (j > len(search_for)*2): # Then it's a search_against forward plot
                motif = search_against_data[against_names[i]]['forward']
                make_logo(motif.copy())
                ax.set_title(f'{for_names[j]} forward')
            elif (j % 2 != 0) and (j > len(search_for)*2): # Then it's a search_against reverse plot
                motif = search_against_data[against_names[i]]['forward']
                make_logo(motif.copy())
                ax.set_title(f'{for_names[j]} reverse')

## Making plots
First, we need to define the filepaths that we're going to plot. First we need to get the RVE elements:

In [48]:
rve_elements = [
    'AT2G46830',
    'AT1G01060',
    'AT5G17300',
    'AT4G01280',
    'AT5G02840',
    'AT3G09600',
    'AT5G52660'
]

In [49]:
rve_path = '/home/farre/Ann/RNAseq/HOMER_analysis/motif_locations/Ath_TF_binding_motifs'
rve_motif_paths = [
    f'{rve_path}/{f}' for f in listdir(rve_path)
    if (f.split('_')[0] in rve_elements) and (f.split('_')[1] == 'ATL')
]

Now we need the elements to search against. Let's start with module 3, which is old module 29:

In [50]:
module3_path = '../data/outputs/ATL_CND_ShD_Leaf_timey_backgrounds_500_start_05May2025/ATL_ShD_Leaf_vs_CND_ShD_Leaf/module_29/ATL/homerResults'
module3_motif_paths = [
    f'{module3_path}/{f}' for f in listdir('../data/outputs/ATL_CND_ShD_Leaf_timey_backgrounds_500_start_05May2025/ATL_ShD_Leaf_vs_CND_ShD_Leaf/module_29/ATL/homerResults')
    if splitext(f)[1] == '.motif' and 'RV' not in f and 'similar' not in f
]
module3_motif_paths.sort(key = lambda x: int(splitext(basename(x))[0][5:]))

Now plot:

In [58]:
plot_multiple_logos(rve_motif_paths, module3_motif_paths)

IndexError: list index out of range